In [3]:
!pip install jaydebeapi JPype1

In [4]:
import ipywidgets as widgets
from IPython.display import display
import re  # Módulo regex para validação de e-mail
from google.colab import files  # Biblioteca para upload do arquivo "ojdbc8.jar"

uploaded = files.upload()

import jaydebeapi

# Referencia e login da base de dados
jdbc_driver_path = 'ojdbc8.jar'
jdbc_url = 'jdbc:oracle:thin:@oracle.fiap.com.br:1521:ORCL'
username = 'rm558303'
password = '161203'

Saving ojdbc8.jar to ojdbc8.jar


In [78]:
# CRUD para lista
#Create
def create_user(nome, user, email, senha, hierarquia):
    try: # Conexão com a base
        conn = jaydebeapi.connect(
            'oracle.jdbc.driver.OracleDriver',
            jdbc_url,
            [username, password],
            jdbc_driver_path
        )
        if validar_email(email) == True:
          curs = conn.cursor()
          # Execução do codigo SQL INSERT para criar usuario
          curs.execute("INSERT INTO usuario (nome, usuario, email, senha, hierarquia) VALUES (?, ?, ?, ?, ?)", (nome, user, email, senha, hierarquia))
          # print("Usuario criado com sucesso.") # Adicione esta linha para debug
    except Exception as e:
        print(f'Erro ao criar cliente: {e}')
    finally:
        if 'curs' in locals():
            curs.close()
        if 'conn' in locals():
            conn.close()
#Read
def read_user():
    try: # Conexão com a base
        conn = jaydebeapi.connect(
            'oracle.jdbc.driver.OracleDriver',
            jdbc_url,
            [username, password],
            jdbc_driver_path
        )
        curs = conn.cursor()
        # Execução do codigo SQL SELECT para printar usuarios
        curs.execute("SELECT * FROM usuario")
        resultados = curs.fetchall()
        return resultados
    except Exception as e:
        return f'Erro ao ler usuario: {e}'
    finally:
        if 'curs' in locals():
            curs.close()
        if 'conn' in locals():
            conn.close()

#Update
def update_user(cliente_id, novo_nome, novo_email, nova_senha):
    try: # Conexão com a base
        conn = jaydebeapi.connect(
            'oracle.jdbc.driver.OracleDriver',
            jdbc_url,
            [username, password],
            jdbc_driver_path
        )
        curs = conn.cursor()
        # Execução do codigo SQL UPDATE para atualizar dados dos usuarios
        curs.execute("UPDATE usuario SET nome = ?, email = ?, senha = ? WHERE id = ?", (novo_nome, novo_email, nova_senha, cliente_id))
        print("Cliente atualizado com sucesso.")
    except Exception as e:
        print(f'Erro ao atualizar cliente: {e}')
    finally:
        if 'curs' in locals():
            curs.close()
        if 'conn' in locals():
            conn.close()
#Delete
def delete_user(cliente_id):
    try: # Conexão com a base
        conn = jaydebeapi.connect(
            'oracle.jdbc.driver.OracleDriver',
            jdbc_url,
            [username, password],
            jdbc_driver_path
        )
        curs = conn.cursor()
        curs.execute("DELETE FROM usuario WHERE id = ?", (cliente_id))
        print("Cliente deletado com sucesso.")
    except Exception as e:
        print(f'Erro ao deletar cliente: {e}')
    finally:
        if 'curs' in locals():
            curs.close()
        if 'conn' in locals():
            conn.close()

In [86]:
output_menu = widgets.Output()

def validar_email(email):
    padrao_email = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
    if re.match(padrao_email, email):
        return True
    return False

def validar_senha(senha):
    if len(senha) >= 5:
        return True
    return False

def cadastrar_usuario(btn):
    nome_completo = new_full_name_input.value
    nome_usuario = new_user_input.value
    email = new_email_input.value
    senha = new_senha_input.value
    hierarquia = hierarquia_dropdown.value

    # Validação dos campos
    if not validar_email(email):
        print("Formato de e-mail inválido. Por favor, insira um e-mail válido.")
        return
    if not validar_senha(senha):
        print("A senha deve ter no mínimo 6 caracteres.")
        return
    if not hierarquia:
        print("Selecione uma hierarquia (Professor ou Aluno).")
        return
    try:
        # Criando o usuário no banco de dados
        create_user(nome_completo, nome_usuario, email, senha, hierarquia)
        print(f"Cadastro de {hierarquia} realizado com sucesso.")
    except Exception as e:
        print(f"Erro ao cadastrar usuário: {e}")

def validar_login(btn):
    user_input_value = user_input.value
    senha_input_value = senha_input.value

    if not validar_email(user_input_value):
        print("Formato de e-mail inválido. Por favor, insira um e-mail válido.")
        return

    try: # Conexão com a base
        conn = jaydebeapi.connect(
            'oracle.jdbc.driver.OracleDriver',
            jdbc_url,
            [username, password],
            jdbc_driver_path
        )
        curs = conn.cursor()
        curs.execute("SELECT id, hierarquia FROM usuario WHERE email = ? AND senha = ?", (user_input_value, senha_input_value))
        resultado = curs.fetchone()

        if resultado:
            usuario_id, hierarquia_usuario = resultado
            print(f"ID do usuario: {usuario_id}.")  # Adicione esta linha para debug
            print(f"Logado com sucesso como {hierarquia_usuario}.")
            mostrar_menu(hierarquia_usuario)
        else:
            print("Usuário ou senha inválidos.")
    except Exception as e:
        print(f'Erro ao realizar login: {e}')
    finally:
        if 'curs' in locals():
            curs.close()
        if 'conn' in locals():
            conn.close()

def listar_usuarios(hierarquia_usuario):
    todos_usuarios = read_user()  # Obtém todos os usuários do banco de dados
    usuarios_visiveis = []

    # Define os níveis de acesso com base na hierarquia
    niveis_acesso = {
        'Administrador': ['Administrador', 'Professor', 'Aluno'],
        'Professor': ['Professor', 'Aluno'],
        'Aluno': ['Aluno']
    }

    # Filtrar os usuários com base na hierarquia e retornar o ID também
    for usuario in todos_usuarios:
        id_usuario, nome_completo, hierarquia = usuario[0], usuario[1], usuario[5]  # ID índice 0, Nome índice 1, Hierarquia índice 5
        if hierarquia in niveis_acesso[hierarquia_usuario]:
            usuarios_visiveis.append((id_usuario, nome_completo, hierarquia))
    return usuarios_visiveis

def mostrar_menu(hierarquia_usuario):
    print(f"\nUsuários visíveis para {hierarquia_usuario}:")
    usuarios = listar_usuarios(hierarquia_usuario)
    for usuario in usuarios:
        id_usuario, nome_completo, hierarquia = usuario
        print(f"{nome_completo} - {hierarquia}")

    # Verifica se o usuário logado é um professor ou administrador
    if hierarquia_usuario in ['Professor', 'Administrador']:
        exibir_aba_gerenciamento(hierarquia_usuario)

def exibir_aba_gerenciamento(hierarquia_usuario):
    # Filtrar os usuários de acordo com a hierarquia do usuário logado
    usuarios_inferiores = [u for u in listar_usuarios(hierarquia_usuario) if u[2] in ['Aluno', 'Professor']]

    if not usuarios_inferiores:
        with output_menu:
            print("Nenhum usuário disponível para modificar.")
        return

    # Ajustar o dropdown para exibir todos os usuários permitidos de acordo com a hierarquia
    usuario_dropdown = widgets.Dropdown(
        options=[(usuario[1], usuario[0]) for usuario in usuarios_inferiores],  # Nome completo como label e ID como valor
        description='Usuário:'
    )

    novo_user_input = widgets.Text(
        description="Novo Usuário:",
        placeholder="Digite o novo nome de usuário"
    )

    novo_email_input = widgets.Text(
        description="Novo Email:",
        placeholder="Digite o novo email"
    )

    nova_senha_input = widgets.Password(
        description="Nova Senha:",
        placeholder="Digite a nova senha"
    )

    botao_atualizar = widgets.Button(
        description="Atualizar",
        button_style="warning"
    )

    output_atualizacao = widgets.Output()

    # Função para realizar a atualização no banco de dados
    def atualizar_usuario(btn):
        with output_atualizacao:
            output_atualizacao.clear_output()
            usuario_id_selecionado = usuario_dropdown.value  # Pega o ID do usuário
            if usuario_id_selecionado:
                novo_user = novo_user_input.value
                novo_email = novo_email_input.value
                nova_senha = nova_senha_input.value

                if novo_user and novo_email and nova_senha:
                    # Atualizar no banco de dados usando o ID do usuário selecionado
                    try:
                        update_user(usuario_id_selecionado, novo_user, novo_email, nova_senha)  # ID do usuário
                        print(f"Usuário {usuario_id_selecionado} atualizado com sucesso.")
                    except Exception as e:
                        print(f"Erro ao atualizar o usuário: {e}")
                else:
                    print("Por favor, preencha todos os campos para atualizar.")

    botao_atualizar.on_click(atualizar_usuario)

    # Exibir os widgets para gerenciamento de usuários
    with output_menu:
        output_menu.clear_output()  # Limpa a saída antes de exibir o dropdown
        display(
            widgets.VBox([
                widgets.HTML(f"<h4>Gerenciar {hierarquia_usuario == 'Administrador' and 'Professores e Alunos' or 'Alunos'}</h4>"),
                usuario_dropdown,
                novo_user_input,
                novo_email_input,
                nova_senha_input,
                botao_atualizar,
                output_atualizacao
            ])
        )

In [87]:
#Widgets
# Login
output_menu = widgets.Output()
user_input = widgets.Text(
    description="Usuário:",
    placeholder="Digite seu usuário"
)

senha_input = widgets.Password(
    description="Senha:",
    placeholder="Digite sua senha"
)

botao_login = widgets.Button(
    description="Login",
    button_style="success"
)

botao_login.on_click(validar_login)

# Cadastro
new_full_name_input = widgets.Text(
    description="Nome Completo:",
    placeholder="Digite seu nome completo"
)

new_user_input = widgets.Text(
    description="Usuário:",
    placeholder="Digite um nome de usuário"
)

new_email_input = widgets.Text(
    description="Email:",
    placeholder="Digite seu email"
)

new_senha_input = widgets.Password(
    description="Senha:",
    placeholder="Digite uma senha"
)

hierarquia_dropdown = widgets.Dropdown(
    options=['Professor', 'Aluno'],
    description='Hierarquia:',
    placeholder='Selecione sua hierarquia'
)

botao_cadastro = widgets.Button(
    description="Cadastrar",
    button_style="info"
)

botao_cadastro.on_click(cadastrar_usuario)

# Gerenciamento

# Menu

In [14]:
display(
    widgets.HTML("<h3>Cadastro</h3>"),
    new_full_name_input,
    new_user_input,
    new_email_input,
    new_senha_input,
    hierarquia_dropdown,
    botao_cadastro
)

HTML(value='<h3>Cadastro</h3>')

Text(value='', description='Nome Completo:', placeholder='Digite seu nome completo')

Text(value='', description='Usuário:', placeholder='Digite um nome de usuário')

Text(value='', description='Email:', placeholder='Digite seu email')

Password(description='Senha:', placeholder='Digite uma senha')

Dropdown(description='Hierarquia:', options=('Professor', 'Aluno'), value='Professor')

Button(button_style='info', description='Cadastrar', style=ButtonStyle())

Cliente criado com sucesso.
Cadastro de Professor realizado com sucesso.
Cliente criado com sucesso.
Cadastro de Professor realizado com sucesso.
Cliente criado com sucesso.
Cadastro de Aluno realizado com sucesso.


In [88]:
display(
    widgets.HTML("<h3>Login</h3>"),
    user_input,
    senha_input,
    botao_login,
    output_menu
)

HTML(value='<h3>Login</h3>')

Text(value='', description='Usuário:', placeholder='Digite seu usuário')

Password(description='Senha:', placeholder='Digite sua senha')

Button(button_style='success', description='Login', style=ButtonStyle())

Output()

ID do usuario: 1.
Logado com sucesso como Professor.

Usuários visíveis para Professor:
guilherme morais - Professor
guilherme morais - Professor
guilherme morais 3 - Aluno


In [89]:
read_user()

[(1, 'guimeb5', 'guimeb', '12345', 'guimeb5@gmail.com', 'Professor'),
 (2, 'guilherme morais', 'guimeb2', '12345', 'guimeb2@gmail.com', 'Professor'),
 (3, 'guilherme morais 3', 'guimeb3', '12345', 'guimeb3@gmail.com', 'Aluno')]